In [ ]:
#orange
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'  #No logging TF

import tensorflow as tf
import numpy as np
import time
from Generator import OmniglotGenerator
from numpy import *;
import random

other_class=0
nb_class =10+other_class
input_size = 784
img_size=(28,28)
batch_size_om = 16
nb_samples_per_class = 17
train_samples_per_class=1

train_samples=17
mb_size = 64

generator1 = OmniglotGenerator(data_folder='/home/pt/test1/data/omtrain', \
                               batch_size=batch_size_om, nb_samples=nb_class,\
                               nb_samples_per_class=nb_samples_per_class,\
                               max_rotation=0., max_shift=0.,img_size=img_size, max_iter=None,\
                               train_samples_per_class=train_samples_per_class)
x_test,y_test=generator1.sample(nb_class)

example_outputs = y_test.reshape(batch_size_om*nb_class*nb_samples_per_class,1)
example_input=x_test.reshape(batch_size_om*nb_class*nb_samples_per_class,784)

#test_others_input = np.zeros((batch_size_om *nb_samples_per_class*other_class,input_size))
#test_others_outputs  =np.zeros((batch_size_om *nb_samples_per_class*other_class,nb_class))

train_input=np.zeros((batch_size_om *train_samples*(nb_class-other_class),input_size))
train_outputs=np.zeros((batch_size_om *train_samples*(nb_class-other_class),nb_class))
jte=0
jtr=0
jto=0
for k in range (nb_class):
    for i in range (batch_size_om *nb_samples_per_class*nb_class):
        if example_outputs[i]==k:
            train_input[jtr]=example_input[i]
            train_outputs[jtr][k]=1
            jtr+=1
            
                

    
    
def getbatchtrain(size):
    batchtrain=np.zeros((size,input_size))
    labeltrain=np.zeros((size,nb_class))
    for i in range(size):
        idx_1 = random.randint(0, batch_size_om *(nb_class-other_class)*train_samples-1)
        batchtrain[i,:] = train_input[idx_1,:]
        labeltrain[i,:]=train_outputs[idx_1,:]
    return batchtrain,labeltrain

def getbatchtrain2(size):
    batchtrain=np.zeros((size,input_size))
    labeltrain=np.zeros((size,nb_class))
    batchtrain2=np.zeros((size,input_size))
    labeltrain2=np.zeros((size,nb_class))
    for i in range(size):
        idx_1 = random.randint(0, batch_size_om *(nb_class-other_class)*train_samples-1)
        jjj=idx_1/(batch_size_om*train_samples)
        idx_2 = random.randint(int(jjj)*(batch_size_om*train_samples),int(jjj+1)*(batch_size_om*train_samples)-1)
        batchtrain[i,:] = train_input[idx_1,:]
        labeltrain[i,:]=train_outputs[idx_1,:]
        batchtrain2[i,:] = train_input[idx_2,:]
        labeltrain2[i,:]=train_outputs[idx_2,:]
    return batchtrain,labeltrain,batchtrain2,labeltrain2

def getbatchtrain3(size):
    batchtrain=np.zeros((size,input_size))
    labeltrain=np.zeros((size,nb_class))
    batchtrain2=np.zeros((size,input_size))
    labeltrain2=np.zeros((size,nb_class))
    batchtrain3=np.zeros((size,input_size))
    labeltrain3=np.zeros((size,nb_class))
    for i in range(size):
        idx_1 = random.randint(0, batch_size_om *(nb_class-other_class)*train_samples-1)
        jjj=idx_1/(batch_size_om*train_samples)
        idx_2 = random.randint(int(jjj)*(batch_size_om*train_samples),int(jjj+1)*(batch_size_om*train_samples)-1)
        idx_3 = random.randint(int(jjj)*(batch_size_om*train_samples),int(jjj+1)*(batch_size_om*train_samples)-1)
        batchtrain[i,:] = train_input[idx_1,:]
        labeltrain[i,:]=train_outputs[idx_1,:]
        batchtrain2[i,:] = train_input[idx_2,:]
        labeltrain2[i,:]=train_outputs[idx_2,:]
        batchtrain3[i,:] = train_input[idx_3,:]
        labeltrain3[i,:]=train_outputs[idx_3,:]
    return batchtrain,labeltrain,batchtrain2,labeltrain2,batchtrain3,labeltrain3

z,_=getbatchtrain(64)
#z,_=getbatchone(10)
z.shape



In [ ]:
#CVAE2
from __future__ import division, print_function, absolute_import

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/pt/test1/data/mnist", one_hot=True)
# Parameters
learning_rate2 = 0.001
num_steps=160000
batch_size= 32
y_dim2=784
y_dim22=10
# Network Parameters
image_dim2= 784 # MNIST images are 28x28 pixels
hidden_dim20 = 512*2
hidden_dim2 = 512
hidden_dim22 = 512
hidden_dim23=256
hidden_dim24=128
hidden_dim25=10

latent_dim2 =20

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

def sample_c(m, n, ind=-1):
    c = np.zeros([m,n])
    for i in range(m):
        if ind<0:
            ind = np.random.randint(10)
        c[i,ind] = 1
    return c



# Variables
weights2 = {
     'encoder_h01': tf.Variable(glorot_init([y_dim2, hidden_dim2])), 
    'encoder_h02': tf.Variable(glorot_init([hidden_dim2, hidden_dim23])), 
    'encoder_h03': tf.Variable(glorot_init([hidden_dim23, hidden_dim24])), 

    'encoder_h1': tf.Variable(glorot_init([image_dim2+hidden_dim24+y_dim22, hidden_dim2])), 
    'z_mean': tf.Variable(glorot_init([hidden_dim2, latent_dim2])),
    'z_std': tf.Variable(glorot_init([hidden_dim2, latent_dim2])),
    'decoder_h01': tf.Variable(glorot_init([y_dim2, hidden_dim2])),
    'decoder_h02': tf.Variable(glorot_init([hidden_dim2, hidden_dim23])),
    'decoder_h03': tf.Variable(glorot_init([hidden_dim23, hidden_dim24])),
   
    'decoder_h1': tf.Variable(glorot_init([latent_dim2+hidden_dim24+y_dim22, hidden_dim22])),
    'decoder_h2': tf.Variable(glorot_init([hidden_dim22, hidden_dim23])),
    'decoder_h3': tf.Variable(glorot_init([hidden_dim23, hidden_dim22])),

    'decoder_out': tf.Variable(glorot_init([hidden_dim22, image_dim2]))
}
biases2 = {
    'encoder_b01': tf.Variable(glorot_init([hidden_dim2])),
    'encoder_b02': tf.Variable(glorot_init([hidden_dim23])),
    'encoder_b03': tf.Variable(glorot_init([hidden_dim24])),

    'encoder_b1': tf.Variable(glorot_init([hidden_dim2])),
    'z_mean': tf.Variable(glorot_init([latent_dim2])),
    'z_std': tf.Variable(glorot_init([latent_dim2])),
    'decoder_b01': tf.Variable(glorot_init([hidden_dim2])),
    'decoder_b02': tf.Variable(glorot_init([hidden_dim23])),
    'decoder_b03': tf.Variable(glorot_init([hidden_dim24])),

    'decoder_b1': tf.Variable(glorot_init([hidden_dim22])),
    'decoder_b2': tf.Variable(glorot_init([hidden_dim23])),
    'decoder_b3': tf.Variable(glorot_init([hidden_dim22])),

    'decoder_out': tf.Variable(glorot_init([image_dim2]))
}

# Building the encoder
input_image2 = tf.placeholder(tf.float32, shape=[None, image_dim2])


y2 = tf.placeholder(tf.float32, shape=[None, y_dim2])
y22 = tf.placeholder(tf.float32, shape=[None, y_dim22])

encoder2 = tf.matmul(y2, weights2['encoder_h01']) + biases2['encoder_b01']
encoder2 = tf.nn.sigmoid(encoder2)
encoder2 = tf.matmul(encoder2, weights2['encoder_h02']) + biases2['encoder_b02']
encoder2 = tf.nn.sigmoid(encoder2)
encoder2 = tf.matmul(encoder2, weights2['encoder_h03']) + biases2['encoder_b03']
encoder2 = tf.nn.sigmoid(encoder2)


encoder2 = tf.matmul(tf.concat([input_image2,encoder2,y22], 1), weights2['encoder_h1']) + biases2['encoder_b1']
encoder2 = tf.nn.tanh(encoder2)


z_mean2 = tf.matmul(encoder2, weights2['z_mean']) + biases2['z_mean']
z_std2 = tf.matmul(encoder2, weights2['z_std']) + biases2['z_std']

# Sampler: Normal (gaussian) random distribution
eps2 = tf.random_normal(tf.shape(z_std2), dtype=tf.float32, mean=0., stddev=1.0,
                       name='epsilon')
z2 = z_mean2 + tf.exp(z_std2 / 2) * eps2

# Building the decoder (with scope to re-use these layers later)
decoder2 = tf.matmul(y2, weights2['decoder_h01']) + biases2['decoder_b01']
decoder2 = tf.nn.sigmoid(decoder2)
decoder2 = tf.matmul(decoder2, weights2['decoder_h02']) + biases2['decoder_b02']
decoder2 = tf.nn.sigmoid(decoder2)
decoder2 = tf.matmul(decoder2, weights2['decoder_h03']) + biases2['decoder_b03']
decoder2 = tf.nn.sigmoid(decoder2)


decoder2 = tf.matmul(tf.concat([z2,decoder2,y22], 1), weights2['decoder_h1']) + biases2['decoder_b1']
decoder2 = tf.nn.tanh(decoder2)
decoder2 = tf.matmul(decoder2, weights2['decoder_h2']) + biases2['decoder_b2']
decoder2 = tf.nn.tanh(decoder2)
decoder2 = tf.matmul(decoder2, weights2['decoder_h3']) + biases2['decoder_b3']
decoder2 = tf.nn.tanh(decoder2)

decoder2 = tf.matmul(decoder2, weights2['decoder_out']) + biases2['decoder_out']
decoder2 = tf.nn.sigmoid(decoder2)

# Define VAE Loss
def vae_loss2(x_reconstructed, x_true):
    # Reconstruction loss
    encode_decode_loss = x_true * tf.log(1e-10 + x_reconstructed) \
                         + (1 - x_true) * tf.log(1e-10 + 1 - x_reconstructed)
    encode_decode_loss = -tf.reduce_sum(encode_decode_loss, 1)
    # KL Divergence loss
    kl_div_loss = 1 + z_std2 - tf.square(z_mean2) - tf.exp(z_std2)
    kl_div_loss = -0.5 * tf.reduce_sum(kl_div_loss, 1)
    return tf.reduce_mean(encode_decode_loss + kl_div_loss)

loss_op2 = vae_loss2(decoder2, input_image2)
optimizer2 = tf.train.RMSPropOptimizer(learning_rate=learning_rate2)
train_op2 = optimizer2.minimize(loss_op2)


In [ ]:
sess = tf.Session()
saver = tf.train.Saver() 
sess.run(tf.global_variables_initializer())
noise_dim=20
for i in range(1, num_steps2+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    #batch_x, _ = mnist.train.next_batch(batch_size)
    #yin=np.zeros((batch_size,y_dim))
    # Train
    
    #x_train1,_,x_train2,_=getbatchtraintest(64)
    #x_train1,x_train2=getbatchtraintest(64)
    #X_mb,y_mb= mnist.train.next_batch(64)
 
    
    if (1):
        #ztest = np.random.uniform(-1., 1., size=[64, noise_dim])
        yset=sample_c(32,y_dim22)
        X_mb,y_mb,X_mb2,y_mb2=getbatchtrain2(32)
 
        feed_dict = {input_image2: X_mb,y2:X_mb2,y22:yset}
        _, l = sess.run([train_op2, loss_op2], feed_dict=feed_dict)
        
    
    if i % 1000 == 0 or i == 1:

        print('Step %i, Loss: %f' % (i, l))
